In [1]:
# Copyright 2020 Bimghi Choi. All Rights Reserved.
# 예측 + 투자전략 시스템

# _*_ coding: utf-8 _*_

from tensorflow import keras
import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]='0, 1'

In [2]:
import util
import preprocess as prepro
import models
import learn
from learn import GenerateResult
import math
from datetime import datetime

In [3]:
#tf.keras.backend.set_floatx('float64')

In [4]:
import gc
gc.collect()

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
#    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')    
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')    
    tf.config.experimental.set_virtual_device_configuration(
        gpus[1],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
  except RuntimeError as e:
    # 프로그램 시작시에 가장 장치가 설정되어야만 합니다
    print(e)

In [5]:
file_name = '~/Data/kospi200f_809_0515.csv'
item_name = 'kospi200f_reinfo_809'
train_start = '2000-01-31'
train_end = '2019-01-02'
test_start = '2019-01-02'
test_end = '2020-05-15'

remove_columns = ['date', '종가']
target_column = '종가'
input_columns = []
target_type = 'rate'

model_name = 'ddaeryuble6'
channel = False

trans_day = 1

target_alpha = 100
future_day = 2
train_end_back = -1
n_timestep = 15
time_interval = 1
input_size = 809
n_unit = 128
batch_size = 35
learning_rate = 0.0005
epochs = 1000

alpha = 0.05
beta = 0.01
gamma = 1

comment = "예측 모델과 best share decision 모델의 reinforcing interaction,  수익률의 합으로 평가, loss=mse"

checkpoint_path = model_name + "/pred"+str(future_day)+"_trans"+str(trans_day)+".ckpt"

In [6]:
dataframe = util.read_datafile(file_name)
df = dataframe.copy()
df['close'] = df[target_column] # 종가 column 추가, 기존 종가는 rate으로 변환 예정
a = []
b = []
#df = prepro.target_conversion(df, target_column, future_day, type=target_type)
for i in range(len(df[target_column]) -future_day):
    df.loc[i, target_column] = ((df.loc[i + future_day, target_column] - df.loc[i, target_column]) 
                                / df.loc[i, target_column]) * target_alpha     
    df.loc[i, '시가'] = df.loc[i + future_day + train_end_back + 1, '시가']
    a.append(max(df.loc[i  + future_day + train_end_back + 1:i + future_day + 1, '고가']))
    b.append(min(df.loc[i  + future_day + train_end_back + 1:i + future_day + 1, '저가']))
for i in range(len(df['종가']) - future_day):
    df.loc[i, '고가'] = a[i]
    df.loc[i, '저가'] = b[i]    

In [7]:
df.head(20)

,date,시가지수(포인트),고가지수(포인트),저가지수(포인트),종가지수(포인트),수익률(%),수익률 (1주)(%),수익률 (1개월)(%),수익률 (3개월)(%),수익률 (6개월)(%),...,주요상품선물_금(선물)($/ounce),주요상품선물_은(선물)($/ounce),주요상품선물_알루미늄(선물)($/ton),주요상품선물_옥수수(최근월물)(￠/bu),대두박(￠/bu),종가,시가,고가,저가,close
0,2000-01-31,-0.218225,-0.493106,-0.400439,-0.223565,0.212872,1.352924,-0.851790,-0.382658,-0.162615,...,-0.853010,0.974513,0.827681,0.373581,0.614542,-0.961538,116.55,120.25,115.95,119.60
1,2000-02-01,-0.213200,-0.230417,-0.373954,-0.595373,-0.407635,1.808646,-1.074203,-1.707405,-0.490731,...,-0.039689,0.193571,1.002058,0.594935,0.706918,3.381014,119.35,123.45,116.80,115.35
2,2000-02-02,-0.208520,-0.448886,-0.367787,-0.200398,0.696677,2.107257,-0.769928,-1.558400,0.074389,...,0.800355,0.031266,0.069626,0.430159,0.725768,3.292528,120.40,123.60,119.75,118.45
3,2000-02-03,-0.204125,-0.216603,-0.062403,-0.021445,0.366314,1.488978,-0.625155,-1.268930,-0.079967,...,1.515992,0.623370,0.378121,0.084769,0.352875,1.677149,122.90,124.10,120.95,119.25
4,2000-02-07,-0.200000,0.378008,0.426748,0.592789,0.949260,1.101574,1.281908,-1.288431,1.273730,...,3.856851,0.568635,-0.322023,0.518339,0.702038,0.653862,122.40,125.05,120.40,122.35
5,2000-02-08,-0.196110,0.353748,0.677387,0.254527,-0.366454,1.138729,0.998930,-1.329397,0.942469,...,2.657684,1.222974,-0.720402,0.474125,0.440887,1.319588,120.85,125.05,119.85,121.25
6,2000-02-09,-0.192447,0.306497,0.819824,0.658684,0.641075,1.076862,1.233568,-1.637689,0.994675,...,3.026664,2.114708,-0.407511,0.622228,0.721716,-1.867641,123.60,124.75,115.25,123.15
7,2000-02-10,0.330923,0.617187,0.552762,0.366383,-0.311193,0.651653,0.275024,-1.607973,0.269679,...,3.316674,1.871659,-0.157431,0.985272,1.379493,-5.575906,118.85,119.00,110.45,122.85
8,2000-02-11,0.977388,0.576306,0.048738,-0.013448,-0.428351,0.326747,0.131530,-1.828185,0.546650,...,2.384344,0.988933,-0.154629,0.473525,0.907189,-8.274721,116.45,116.80,105.25,120.85
9,2000-02-14,-0.649462,-0.806483,-0.954529,-1.222378,-1.533955,-1.217328,-0.075927,-2.486079,0.140712,...,1.935983,0.085505,-0.246361,0.529501,0.923675,-1.422414,112.05,114.35,105.25,116.00


In [8]:
@tf.function
def loss_fn_model1(train_y, preds):
    
    loss = []
    for i in range(preds.shape[0]):
        for j in range(preds.shape[1]):
                loss.append(-train_y[i, j. 0]*tf.math.log(pred[i, j, 0]))
    loss = tf.reshape(loss, [preds.shape[0], preds.shape[1], preds.shape[2]])
    
    return loss


In [9]:
@tf.function
def loss_fn_model2(train_y, preds):
    
    # reaward 계산
    model1_preds = train_y[:, :, 0]
    rates = train_y[:, :, 1]
    profits = tf.math.multiply(tf.math.sign(model1_preds), rates)
    next_profits = tf.reshape(tf.concat([profits[:, 1:, -1], profits[:, -1, -1]], 1), [train_y.shape[0], train_y,shape[1], train_y.shape[2]])
    q_targets = profits + alpha*next_profits
    
   
    return keras.losses.MSE(q_targets, preds)

In [10]:
def test(m1, m2, test_x, test_y):
    
    preds = tf.cast(m1(test_x), dtype=tf.float64)

    # model2 input 생성
    test_x_m2 = tf.concat([test_x, preds], 2)
    y = tf.reshape(m2(test_x_m2)[:, -1, :], [-1])

    return y

In [11]:
#@tf.function
def gradient1(model1, model2, input_data, output_data):
    with tf.GradientTape() as tape:
        loss = loss_fn_model1(model1, model2, input_data, output_data)
    return tape.gradient(loss, model1.trainable_variables)
#@tf.function
def gradient2(model1, model2, input_data, output_data):
    with tf.GradientTape() as tape:
        loss = loss_fn_model2(model1, model2, input_data, output_data)
    return tape.gradient(loss, model2.trainable_variables)

In [12]:
#strategy = tf.distribute.MirroredStrategy()
#with strategy.scope():
model1 = models.LSTM_tanh(n_timestep,input_size,n_unit,regularizers_alpha=0.01,drop_rate=0.5)
#model2 = models.LSTM_sigm(n_timestep,2,10,regularizers_alpha=0.01,drop_rate=0)

model2 = tf.keras.Sequential([
    tf.keras.Input(shape=(n_timestep, 2)),
    tf.keras.layers.LSTM(256, return_sequences=True, kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.LSTM(128, return_sequences=True, kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(1),
])
model2.summary()

    #cp_callback = tf.keras.callbacks.ModelCheckpoint(
    #    checkpoint_path, verbose=1, save_weights_only=True,
        # 다섯 번째 에포크마다 가중치를 저장합니다
    #    save_freq=5)

model1.compile(optimizer='adam',
              loss=loss_fn_model1)
                  #callbacks=[cp-callback]
              #metrics=['accuracy'])

model2.compile(optimizer='adam',
              loss=loss_fn_model2)
                  #callbacks=[cp-callback]
              #metrics=['accuracy'])        

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 15, 256)           1092608   
_________________________________________________________________
lstm_4 (LSTM)                (None, 15, 128)           197120    
_________________________________________________________________
dense_1 (Dense)              (None, 15, 1)             129       
Total params: 1,289,857
Trainable params: 1,289,857
Non-trainable params: 0
_________________________________________________________________


In [13]:
current_train_start = df.loc[prepro.date_to_index(df, train_start), 'date']
current_train_end = df.loc[prepro.date_to_index(df, train_end), 'date']
current_test_start = df.loc[prepro.date_to_index(df, test_start), 'date']
current_test_end = df.loc[prepro.date_to_index(df, test_start) + trans_day - 1, 'date']


#  각 transfer 구간의 예측값들을 합치기 위하여
test_prediction1 = []
test_prediction2 = []

print("실험 시작")
print(datetime.now().strftime("%Y-%m-%d:%H:%M:%S"))

while True:

    gc.collect()
    
    #model2 data
    train_data, test_data = prepro.get_train_test_data(df, target_column, remove_columns, 
                                                           current_train_start, current_train_end,
                                                           current_test_start, current_test_end,
                                                           future_day, n_timestep, time_interval)


    train_x, train_y = prepro.get_LSTM_dataset(train_data, n_timestep, time_interval, input_size, future_day)
    
    # randomly selected  step size
    #drop_idx = np.random.randint(int(train_y.shape[1]/2))
    #train_x[:, :drop_idx, :] = 0
    #train_y[:, :drop_idx, :] = 0    
    
    train_x = train_x[:train_end_back]
    train_y = train_y[:train_end_back]
    
    test_x, test_y = prepro.get_LSTM_dataset(test_data, n_timestep, time_interval, input_size, future_day)
    test_y_target = np.expand_dims(test_y[:, :, 4], axis=2)    
    
    # train, validation set 분리하여 train1은 예측모델로 train2는 손절값 학습 모델로 사용
    train1_x, train2_x, train1_y, train2_y = train_test_split(train_x, train_y, test_size=0.5)    
    
    updown = np.sign(test_y[:, -1, 4]).reshape((-1))
    
    # random batch_size
    #batch_size = np.random.randint(10, 50)    

    # model1 training
    train1_y_target = model2(tf.concat(train_x, model1(train_x), 2))
    #early_stopping1 = tf.keras.callbacks.EarlyStopping(patience=2, verbose=1)
    model1.fit(train1_x, train1_y_target, batch_size=batch_size, epochs=1)#, callbacks=[early_stopping1], validation_data=(test_x, test_y_target))
    
    
    # model2 training
    train2_rates = tf.expand_dims(train2_y[:, :, 4], axis=2)
    train2_preds = model1(train2_x)
    train2_x = tf.concat([train2_preds, train2_rates], 2)
    model2.fit(train2_x, train2_x, batch_size=batch_size, epochs=1)

    print('test dates ' + current_test_start + "~" + current_test_end)

    # prediction1 accuracy
    prediction1 = model1.predict(test_x)[:, -1, 0].reshape(-1)
    temp = tf.math.multiply(updown, prediction1)
    accu = tf.reduce_sum(list(map(lambda x: 1 if x > 0 else 0, temp)))/len(temp)
    print('prediction1 accuracy = ', accu)

    # reinforced prediction2
    prediction2 = test(model1, model2, test_x, test_y)    
    print('optimal shares = ', prediction2)

    test_prediction1.append(prediction1)
    test_prediction2.append(prediction2)

    # escape from while
    if current_test_end == test_end:
     break

    #train, start dates shift
    current_train_end = df.loc[prepro.date_to_index(df, current_train_end) + trans_day, 'date']
    current_train_start = df.loc[prepro.date_to_index(df, current_train_end) - 1000, 'date']
    current_test_start = df.loc[prepro.date_to_index(df, current_test_start) + trans_day, 'date']
    if prepro.date_to_index(df, test_end) - prepro.date_to_index(df, current_test_start) < trans_day:
        current_test_end = test_end
    else:
        current_test_end = df.loc[prepro.date_to_index(df, current_test_end) + trans_day, 'date']

실험 시작
2020-07-31:22:01:21
Train on 2327 samples, validate on 1 samples
Epoch 1/3
2327/2327 [==============================] - 16s 7ms/sample - loss: 24.8323 - val_loss: 13.9290
Epoch 2/3
2327/2327 [==============================] - 1s 461us/sample - loss: 12.2158 - val_loss: 7.4788
Epoch 3/3
2327/2327 [==============================] - 1s 357us/sample - loss: 6.9728 - val_loss: 5.0209

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Train on 2328 samples
Epoch 1/3
2328/2328 [==============================] - 4s 2ms/sample - loss: 14.3273
Epoch 2/3
232

Train on 492 samples
Epoch 1/3
492/492 [==============================] - 0s 507us/sample - loss: 1.2494e-05
Epoch 2/3
492/492 [==============================] - 0s 504us/sample - loss: 7.2726e-06
Epoch 3/3
492/492 [==============================] - 0s 497us/sample - loss: 4.1935e-06
test dates 2019-01-14~2019-01-14
prediction1 accuracy =  tf.Tensor(0.0, shape=(), dtype=float64)
optimal shares =  tf.Tensor([-2.4116871e-05], shape=(1,), dtype=float32)
Train on 492 samples, validate on 1 samples
Epoch 1/3
492/492 [==============================] - 0s 535us/sample - loss: 0.4730 - val_loss: 0.6271
Epoch 2/3
492/492 [==============================] - 0s 492us/sample - loss: 0.4655 - val_loss: 0.6122
Epoch 3/3
492/492 [==============================] - 0s 536us/sample - loss: 0.4381 - val_loss: 0.5725
Train on 492 samples
Epoch 1/3
492/492 [==============================] - 0s 449us/sample - loss: 2.3952e-06
Epoch 2/3
492/492 [==============================] - 0s 482us/sample - loss: 1.3545

492/492 [==============================] - 0s 386us/sample - loss: 2.2632e-09
Epoch 2/3
492/492 [==============================] - 0s 492us/sample - loss: 1.7130e-09
Epoch 3/3
492/492 [==============================] - 0s 528us/sample - loss: 1.5322e-09
test dates 2019-02-14~2019-02-14
prediction1 accuracy =  tf.Tensor(0.0, shape=(), dtype=float64)
optimal shares =  tf.Tensor([-1.2146251e-05], shape=(1,), dtype=float32)
Train on 492 samples, validate on 1 samples
Epoch 1/3
492/492 [==============================] - 0s 505us/sample - loss: 0.2055 - val_loss: 0.2827
Epoch 2/3
492/492 [==============================] - 0s 374us/sample - loss: 0.2017 - val_loss: 0.2999
Epoch 3/3
492/492 [==============================] - 0s 326us/sample - loss: 0.1832 - val_loss: 0.3241
Epoch 00003: early stopping
Train on 492 samples
Epoch 1/3
492/492 [==============================] - 0s 276us/sample - loss: 2.7332e-09
Epoch 2/3
492/492 [==============================] - 0s 286us/sample - loss: 4.0020e-0

492/492 [==============================] - 0s 552us/sample - loss: 2.8544e-09
Epoch 2/3
492/492 [==============================] - 0s 441us/sample - loss: 2.7043e-09
Epoch 3/3
492/492 [==============================] - 0s 500us/sample - loss: 2.4866e-09
test dates 2019-03-15~2019-03-15
prediction1 accuracy =  tf.Tensor(1.0, shape=(), dtype=float64)
optimal shares =  tf.Tensor([-8.117962e-06], shape=(1,), dtype=float32)
Train on 492 samples, validate on 1 samples
Epoch 1/3
492/492 [==============================] - 0s 508us/sample - loss: 0.1287 - val_loss: 0.4138
Epoch 2/3
492/492 [==============================] - 0s 579us/sample - loss: 0.1307 - val_loss: 0.3780
Epoch 3/3
492/492 [==============================] - 0s 518us/sample - loss: 0.1399 - val_loss: 0.4362
Train on 492 samples
Epoch 1/3
492/492 [==============================] - 0s 541us/sample - loss: 3.8172e-09
Epoch 2/3
492/492 [==============================] - 0s 351us/sample - loss: 4.4430e-09
Epoch 3/3
492/492 [========

492/492 [==============================] - 0s 484us/sample - loss: 5.1099e-09
Epoch 3/3
492/492 [==============================] - 0s 513us/sample - loss: 4.1945e-09
test dates 2019-04-12~2019-04-12
prediction1 accuracy =  tf.Tensor(1.0, shape=(), dtype=float64)
optimal shares =  tf.Tensor([-6.061828e-06], shape=(1,), dtype=float32)
Train on 492 samples, validate on 1 samples
Epoch 1/3
492/492 [==============================] - 0s 565us/sample - loss: 0.1134 - val_loss: 0.3012
Epoch 2/3
492/492 [==============================] - 0s 505us/sample - loss: 0.1179 - val_loss: 0.2830
Epoch 3/3
492/492 [==============================] - 0s 492us/sample - loss: 0.1143 - val_loss: 0.2330
Train on 492 samples
Epoch 1/3
492/492 [==============================] - 0s 501us/sample - loss: 4.9757e-09
Epoch 2/3
492/492 [==============================] - 0s 355us/sample - loss: 3.5329e-09
Epoch 3/3
492/492 [==============================] - 0s 294us/sample - loss: 6.2214e-09
test dates 2019-04-15~2019-

...

Train on 492 samples, validate on 1 samples
Epoch 1/3
492/492 [==============================] - 0s 519us/sample - loss: 0.0957 - val_loss: 0.5365
Epoch 2/3
492/492 [==============================] - 0s 491us/sample - loss: 0.0857 - val_loss: 0.5323
Epoch 3/3
492/492 [==============================] - 0s 465us/sample - loss: 0.0787 - val_loss: 0.3241
Train on 492 samples
Epoch 1/3
492/492 [==============================] - 0s 426us/sample - loss: 3.3340e-06
Epoch 2/3
492/492 [==============================] - 0s 375us/sample - loss: 3.3659e-06
Epoch 3/3
492/492 [==============================] - 0s 384us/sample - loss: 3.4063e-06
test dates 2020-02-07~2020-02-07
prediction1 accuracy =  tf.Tensor(1.0, shape=(), dtype=float64)
optimal shares =  tf.Tensor([-5.9868777e-05], shape=(1,), dtype=float32)
Train on 492 samples, validate on 1 samples
Epoch 1/3
492/492 [==============================] - 0s 515us/sample - loss: 0.0895 - val_loss: 1.2653
Epoch 2/3
492/492 [==========================

492/492 [==============================] - 0s 521us/sample - loss: 0.0730 - val_loss: 0.3834
Epoch 2/3
492/492 [==============================] - 0s 508us/sample - loss: 0.0684 - val_loss: 0.3482
Epoch 3/3
492/492 [==============================] - 0s 455us/sample - loss: 0.0694 - val_loss: 0.3696
Train on 492 samples
Epoch 1/3
492/492 [==============================] - 0s 385us/sample - loss: 2.8748e-06
Epoch 2/3
492/492 [==============================] - 0s 465us/sample - loss: 3.3334e-06
Epoch 3/3
492/492 [==============================] - 0s 460us/sample - loss: 3.0638e-06
test dates 2020-02-21~2020-02-21
prediction1 accuracy =  tf.Tensor(1.0, shape=(), dtype=float64)
optimal shares =  tf.Tensor([-8.099523e-05], shape=(1,), dtype=float32)
Train on 492 samples, validate on 1 samples
Epoch 1/3
492/492 [==============================] - 0s 379us/sample - loss: 0.0689 - val_loss: 2.0654
Epoch 2/3
492/492 [==============================] - 0s 410us/sample - loss: 0.0670 - val_loss: 1.97

492/492 [==============================] - 0s 359us/sample - loss: 0.0919 - val_loss: 7.2194
Train on 492 samples
Epoch 1/3
492/492 [==============================] - 0s 502us/sample - loss: 3.3314e-07
Epoch 2/3
492/492 [==============================] - 0s 377us/sample - loss: 3.1099e-07
Epoch 3/3
492/492 [==============================] - 0s 412us/sample - loss: 2.8976e-07
test dates 2020-03-20~2020-03-20
prediction1 accuracy =  tf.Tensor(0.0, shape=(), dtype=float64)
optimal shares =  tf.Tensor([-0.00028964], shape=(1,), dtype=float32)
Train on 492 samples, validate on 1 samples
Epoch 1/3
492/492 [==============================] - 0s 518us/sample - loss: 0.1156 - val_loss: 9.1526
Epoch 2/3
492/492 [==============================] - 0s 487us/sample - loss: 0.1088 - val_loss: 6.5355
Epoch 3/3
492/492 [==============================] - 0s 482us/sample - loss: 0.1026 - val_loss: 5.2511
Train on 492 samples
Epoch 1/3
492/492 [==============================] - 0s 429us/sample - loss: 2.70

Train on 492 samples
Epoch 1/3
492/492 [==============================] - 0s 433us/sample - loss: 1.0965e-08
Epoch 2/3
492/492 [==============================] - 0s 429us/sample - loss: 9.8908e-09
Epoch 3/3
492/492 [==============================] - 0s 402us/sample - loss: 1.1057e-08
test dates 2020-04-20~2020-04-20
prediction1 accuracy =  tf.Tensor(1.0, shape=(), dtype=float64)
optimal shares =  tf.Tensor([3.9516017e-05], shape=(1,), dtype=float32)
Train on 492 samples, validate on 1 samples
Epoch 1/3
492/492 [==============================] - 0s 494us/sample - loss: 0.1433 - val_loss: 1.2452
Epoch 2/3
492/492 [==============================] - 0s 458us/sample - loss: 0.2179 - val_loss: 1.3636
Epoch 3/3
492/492 [==============================] - 0s 474us/sample - loss: 0.2026 - val_loss: 1.0992
Train on 492 samples
Epoch 1/3
492/492 [==============================] - 0s 428us/sample - loss: 9.6212e-09
Epoch 2/3
492/492 [==============================] - 0s 404us/sample - loss: 1.0295e

Train on 492 samples
Epoch 1/3
492/492 [==============================] - 0s 415us/sample - loss: 1.2300e-08
Epoch 2/3
492/492 [==============================] - 0s 361us/sample - loss: 1.0883e-08
Epoch 3/3
492/492 [==============================] - 0s 383us/sample - loss: 1.2612e-08
test dates 2020-05-07~2020-05-07
prediction1 accuracy =  tf.Tensor(1.0, shape=(), dtype=float64)
optimal shares =  tf.Tensor([5.2969262e-05], shape=(1,), dtype=float32)
Train on 492 samples, validate on 1 samples
Epoch 1/3
492/492 [==============================] - 0s 479us/sample - loss: 0.2310 - val_loss: 0.3562
Epoch 2/3
492/492 [==============================] - 0s 494us/sample - loss: 0.1652 - val_loss: 0.4099
Epoch 3/3
492/492 [==============================] - 0s 434us/sample - loss: 0.1303 - val_loss: 0.3334
Train on 492 samples
Epoch 1/3
492/492 [==============================] - 0s 391us/sample - loss: 1.2666e-08
Epoch 2/3
492/492 [==============================] - 0s 391us/sample - loss: 1.3185e

In [14]:
print("실험 종료")
print(datetime.now().strftime("%Y-%m-%d:%H:%M:%S"))

t1 = np.concatenate(test_prediction1)
t2 = np.concatenate(test_prediction2)

실험 종료
2020-08-01:01:03:51


In [15]:
#t2

In [16]:
#test_prediction = np.concatenate(test_prediction, axis=0).reshape((-1, n_timestep, 1))
#train_prediction = learn.predict_batch_test(model1, train_x[:batch_size], len(train_x[:batch_size]))

# 원 지수, 날짜는 target conversion이 되기 전 dataframe에서 가져옴
test_dates, test_base_prices, train_dates, train_base_prices = prepro.get_test_dates_prices(dataframe, test_start, test_end,
                                                      train_start, train_end, n_timestep, time_interval, future_day, target_column)

# 전체 test_oouput 생성
_, test_data = prepro.get_train_test_data(df, target_column, remove_columns,
                                                   train_start, train_end,
                                                   test_start, test_end,
                                                   future_day, n_timestep, time_interval)
_, test_y = prepro.get_LSTM_dataset(test_data, n_timestep, time_interval, input_size, future_day)

In [17]:
# 원 지수, 날짜는 target conversion이 되기 전 dataframe에서 가져옴 - 종가를 test base price로 하는 경우
result = GenerateResult(t1, t2, test_y[:, -1, 4].reshape(-1), test_dates, n_timestep, future_day, trans_day)

#result.extract_last_output()
result.convert_price(test_base_prices,conversion_type=target_type)

# 손익 계산을 위한 데이터 정리
test_start_index = prepro.date_to_index(df, test_start)
test_end_index = prepro.date_to_index(df, test_end)
test_open_prices = list(map(float, df.loc[test_start_index - future_day: test_end_index - future_day, '시가']))
test_high_prices = list(map(float, df.loc[test_start_index - future_day: test_end_index - future_day, '고가']))
test_low_prices = list(map(float, df.loc[test_start_index - future_day: test_end_index - future_day, '저가']))

# 예측 손익 계산
profits = np.zeros(len(test_dates))
for i in range(len(test_dates)):
    if result.test_predict_price[i]-test_open_prices[i] > 0:
        profits[i] = result.test_output_price[i] - test_open_prices[i]
    else:
        profits[i] = test_open_prices[i] - result.test_output_price[i]

# 투자 비율 반영 손익 계산
profits2 = np.zeros(len(test_dates))
for i in range(len(test_dates)):
    profits2[i] = (result.test_output_price[i] - test_open_prices[i])*t2[i]


In [ ]:
# 종가 대비 에측 확률 계싼
updown = np.sign(test_y[:, -1, 4]).reshape((-1))
temp = tf.math.multiply(updown, t1.reshape((-1)))
accu = tf.reduce_sum(list(map(lambda x: 1 if x > 0 else 0, temp)))/len(temp)
print('종가 대비 accuracy = ', accu)

# 시가 대비 예측 확률1
cnt  = 0
for i in range(len(profits)):
    if profits[i] > 0:
        cnt += 1
accu = cnt / len(profits1)        
print('시가 대비 accuracy1 = ', accu)

# 시가 대비 예측 확률2
cnt  = 0
for i in range(len(profits)):
    if profits2[i] > 0:
        cnt += 1
accu = cnt / len(profits2)        
print('시가 대비 accuracy2 = ', accu)

In [ ]:
print(tf.shape(result.test_output_price))
print(tf.shape(result.test_predict_price))
print(tf.shape(test_low_prices))
print(tf.shape(profits))
print(tf.shape(profits2))

In [ ]:
result.evaluation()
result.table(test_open_prices, test_high_prices, test_low_prices, profits, profits2)
result.save_result(model_name,item_name,n_unit,target_type,batch_size,n_timestep,time_interval,epochs,str(alpha),comment)
result.save_visualization()
result.save_model(model2)